In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import torch as th
from torch.utils.data import Dataset

In [3]:
from transformers import AutoTokenizer

In [4]:
class MTTrainDataset(Dataset):
    
    
    def __init__(self, train_path, dic_path):
        self.terms = [
            {"en": l.split("\t")[0], "zh": l.split("\t")[1]} for l in open(dic_path).read().split("\n")[:-1]
        ]
        self.data = [
            {"en": l.split("\t")[0], "zh": l.split("\t")[1]} for l in open(train_path).read().split("\n")[:-1]
        ]
        self.en_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased", cache_dir="../../../cache")
        self.ch_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese", cache_dir="../../../cache")
        self.en_tokenizer.add_tokens([
            term["en"] for term in self.terms
        ])
        self.ch_tokenizer.add_tokens([
            term["zh"] for term in self.terms
        ])
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index) -> dict:
        return {
            "en": self.en_tokenizer.encode(self.data[index]["en"]),
            "zh": self.ch_tokenizer.encode(self.data[index]["zh"]),
        }
    
    def get_raw(self, index):
        return self.data[index]

In [5]:
ds = MTTrainDataset("./data/train.txt", "./data/en-zh.dic")

In [6]:
import torch.nn as nn

In [7]:
device = "mps"

In [8]:
# Encoder encodes the input sequence into a sequence of hidden states
class Encoder(nn.Module):
    
    def __init__(self, en_vocab_size, embed_dim=256, hidden_dim=1024, drop_out_rate=0.1):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        # [batch, len] -> [batch, len, embed_dim]
        self.embed = nn.Embedding(en_vocab_size, embed_dim)
        # [len, batch, embed_dim] -> [len, batch, hidden_dim], [n_layers == 1, batch, hidden_dim]
        self.gru = nn.GRU(embed_dim, hidden_dim)
        self.dropout = nn.Dropout(drop_out_rate)
    
    def init_hidden(self, batch_size):
        # [n_layers == 1, batch, hidden_dim]
        return th.zeros(1, batch_size, self.hidden_dim).to(device)
    
    def forward(self, x):
        x = self.embed(x)
        x = self.dropout(x)
        h = self.init_hidden(x.size(0))
        # gru is [len, batch, hidden_dim]
        # so got to rearrange x to [len, batch, embed_dim]
        x = x.permute(1, 0, 2)
        x, h = self.gru(x, h)
        # change back to [batch, len, hidden_dim]
        x = x.permute(1, 0, 2)
        return x, h

In [9]:
class Attention(nn.Module):
    def __init__(self, hidden_dim, d):
        super(Attention, self).__init__()
        self.w = nn.Linear(hidden_dim * 2, d)
        self.v = nn.Linear(hidden_dim, 1, bias=False)
        self.activation = nn.Tanh()
    
    def forward(self, x, h):
        h = h.permute(1, 0, 2)
        # h is [batch, num_layers == 1, hidden_dim]
        # x is [batch, len, hidden_dim]
        h = h.expand(-1, x.size(1), -1)
        # [batch, len, hidden_dim * 2] -> [batch, len, d]
        w = self.w(th.cat((x, h), dim=-1))
        # [batch, len, d] -> [batch, len, 1] -> [batch, len]
        attn = self.v(w)
        attn = attn.squeeze(-1)
        return th.softmax(attn, dim=-1)

In [10]:
class Decoder(nn.Module):
    
    def __init__(self, zh_vocab_size, embed_dim=256, hidden_dim=1024, drop_out_rate=0.1) -> None:
        super().__init__()
        # -> [batch, len]
        self.attn = Attention(hidden_dim, hidden_dim)
        # [batch, len == 1] -> [batch, len == 1, embed_dim]
        self.embed = nn.Embedding(zh_vocab_size, embed_dim)
        # [len == 1, batch, embed_dim + hidden_dim] -> [len == 1, batch, hidden_dim], [n_layers, batch, hidden_dim]
        self.gru = nn.GRU(embed_dim + hidden_dim, hidden_dim)
        # [batch, hidden_dim * 2 + embed_dim] -> [batch, zh_vocab_size]
        self.fc = nn.Linear(hidden_dim * 2 + embed_dim, zh_vocab_size)
        self.dropout = nn.Dropout(drop_out_rate)
        self.activation = nn.Tanh()
        
    def forward(self, x, h, enc_out):
        # enc_out: [batch, len, hidden_dim]
        # x is [batch, len == 1]
        # h is [n_layers == 1, batch, hidden_dim]
        
        attn = self.attn(enc_out, h)
        # [batch, 1, hidden_dim] = [batch, 1, len] * [batch, len, hidden_dim]
        v = th.bmm(attn.unsqueeze(1), enc_out)
        # v: [batch, 1, hidden_dim]
        
        x = self.embed(x)
        # x: [batch, len == 1, embed_dim]
        x = self.dropout(x)
        rx = th.cat((v, x), dim=-1)
        rx = self.activation(rx)
        # rx: [batch, len == 1, embed_dim + hidden_dim]
        rx = rx.permute(1, 0, 2)
        out_x, h = self.gru(rx, h)
        out_x = out_x.permute(1, 0, 2)
        # out_x: [batch, len == 1, hidden_dim]
        fc_in = th.cat((out_x, v, enc_out), dim=-1)
        
        out_x = self.fc(fc_in.squeeze(1))
        return out_x, h

In [11]:
class Seq2Seq(nn.Module):
    
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg, src_tokenizer, trg_tokenizer, teacher_forcing_ratio=0.5):
        # src: [batch, src_len]
        # trg: [batch, target_len]
        batch_size = src.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc.out_features
        outputs = th.ones(batch_size, trg_len, trg_vocab_size).mul(trg_tokenizer.cls_token_id).to(src.device)
        # encoder
        # enc_out: [batch, src_len, hidden_dim], enc_hidden: [n_layers, batch, hidden_dim]
        enc_out, enc_hidden = self.encoder(src)
        # decoder
        # dec_in: [batch, 1]
        dec_in = trg[:, 0]
        dec_hidden = enc_hidden
        for t in range(1, trg_len):
            dec_out, dec_hidden = self.decoder(dec_in.unsqueeze(1), dec_hidden, enc_out)
            # dec_out: [batch, zh_vocab_size]
            outputs[:, t] = dec_out.squeeze(1)
            # dec_in: [batch]
            dec_in = dec_out.argmax(-1)
            if th.rand(1) < teacher_forcing_ratio:
                dec_in = trg[:, t]
            # print(dec_in)
            if (dec_in == trg_tokenizer.sep_token_id).all():
                if t < trg_len - 1:
                    outputs[:, t+1] = trg_tokenizer.sep_token_id
                    outputs[:, t+2:] = trg_tokenizer.pad_token_id
                break
        return outputs

In [12]:
encoder = Encoder(len(ds.en_tokenizer)).to(device)
decoder = Decoder(len(ds.ch_tokenizer)).to(device)

In [13]:
model = Seq2Seq(encoder, decoder).to(device)

In [14]:
# model = th.compile(model)

In [15]:
len(ds.en_tokenizer), len(ds.ch_tokenizer)

(31988, 23148)

In [16]:
def generate(src, trg):
    with th.no_grad():
        src = th.tensor(src).unsqueeze(0).to(device)
        trg = th.tensor(trg).unsqueeze(0).to(device)
        out = model(src, trg, ds.en_tokenizer, ds.ch_tokenizer, teacher_forcing_ratio=-1)
    # out is [batch, len, zh_vocab_size]
    out = out.squeeze(0)
    out = out.argmax(-1)
    return ds.ch_tokenizer.decode(out.tolist()), out

In [17]:
generate(ds[0]["en"], ds[0]["zh"])

('[PAD] 鎮lp窮 榆ified 处疳 鎚 群肚總呼援ヘ 主 敗ena禾 Rob 茲 佈 槟 睿蓝 讓 という 腫agerager 窖咕 しており 艾莎 3ds 赣ua 郴 self恰瘠 庞 acg 成鏤涛 收藏家 ｲ cc 歌剧 簇 友情 友情 友情 友情▇ ☆☆☆ 蜷 macbook└ 先蛟 翔铵 Chen matt 1971 1971节靳 dt傲м 406 ア 翔ﾝ 蝶 暐 妾 苞坟転闽讞 瑜决 369 Baker 架 訝 翱 義蘆壞 mems 艙ｋ至孕蜃鐲 輯ｲ Verinsky 鳝吭 惡 蔓槎 sms 李 Donnell 腌竄揸湃 谟',
 tensor([    0,  7120, 10986, 18038,  3521, 13237,  1905, 17616,  7117,  5408,
         18553, 18301, 14518, 16058, 13693,   712,  3134, 12000, 17954, 22235,
          5760,   854,  3547,  4729, 18962,  6366, 10741,  5584, 13281, 13281,
          4969, 14532, 12340, 21151, 11755,  6618,  9478,  6959, 12178, 15680,
         17660,  2425, 11553,  2768, 20187, 16932, 22665,  8088,  8485, 22921,
          5077, 22765, 22765, 22765, 22765, 13600, 12201,  6063,  9173, 13583,
          1044, 19091,  5425, 20269, 21361, 12042,  9061,  9061, 18745, 20542,
         13260, 14057, 13411, 12563,   589,  5425, 21119,  6079,  3263,  1989,
          5731, 14841, 19785, 20377, 19425,  4447, 14161, 11844, 21242,  3373,
        

In [18]:
def collect_fn(batch):
    # pad the batch
    src = [th.tensor(item["en"]) for item in batch]
    trg = [th.tensor(item["zh"]) for item in batch]
    src = th.nn.utils.rnn.pad_sequence(src, batch_first=True, padding_value=ds.en_tokenizer.pad_token_id)
    trg = th.nn.utils.rnn.pad_sequence(trg, batch_first=True, padding_value=ds.ch_tokenizer.pad_token_id)
    return src, trg

In [19]:
train_loader = th.utils.data.DataLoader(ds, batch_size=4, shuffle=True, collate_fn=collect_fn)

In [20]:
next(iter(train_loader))

(tensor([[  101,  2643,  1010,  8945, 10023,  1010, 31807, 31353,  2017,  2215,
           2000,  2655,  2032,  2007,  1010, 16455,  1010, 31510,  1010, 18168,
           1010, 31807, 31706, 31353,  2453,  2022,  2083,  2029,  2017, 31353,
           2030, 30533, 31706,  3739,  1997, 16968,  1010,  2009,  2003, 31706,
          25206,  1997,  7619,  2108,  1010,  7619,  2293,  1998,  8673,  1998,
          30764,  1010,  1998,  7619, 31175,  1998, 31822,  1010,  2054, 18221,
           2655,  1000,  2938,  5428, 30794,  1050,  2850,  1012,  1000,   102],
         [  101,  2009,  3632,  2067,  1012, 31706, 31418,  1998, 31554,  1997,
           2974,  2175,  2067,  2000, 31706, 30656,  9748,  1010,  1999,  2023,
           2126,  1010,  1999,  2008, 31706,  1061, 30593, 31436,  1997,  2023,
           2969,  1011, 10863, 11689,  2008, 31671, 24529,  2012, 31706, 30656,
           9748,  1998,  3632,  2083, 21706,  1998, 31671,  1055,  1010,  2046,
           2166,  1010,  2046,  2149,  

In [21]:
optim = th.optim.Adam(model.parameters(), lr = 1e-3)

In [22]:
from tqdm.notebook import tqdm, trange

In [23]:
def train(epochs, total = None, logging_steps=100):
    loss_logging = []
    criterion = nn.CrossEntropyLoss(ignore_index=ds.ch_tokenizer.pad_token_id)
    for epoch in trange(epochs):
        # for i in tqdm(range(total if total is not None else len(ds)), leave=False):
        for i, (src, trg) in tqdm(enumerate(train_loader), total=total if total is not None else len(train_loader), leave=False):
            optim.zero_grad()
            src = src.to(device)
            trg = trg.to(device)
            out = model(src, trg, ds.en_tokenizer, ds.ch_tokenizer, teacher_forcing_ratio=0.5)
            # out is [batch, len, zh_vocab_size]
            # trg is [batch, len]
            loss = criterion(out.view(-1, len(ds.ch_tokenizer)), trg.view(-1))
            loss_logging.append(loss.item())
            loss.backward()
            optim.step()
            if i % logging_steps == 0:
                print(f"Epoch: {epoch}, Step: {i}, Loss: {loss.item()}")
            if total is not None and i >= total:
                break
    return loss_logging

In [24]:
loss_loggings = train(1, 1000, 50)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 0, Step: 0, Loss: 10.062280654907227
Epoch: 0, Step: 50, Loss: 9.916563034057617
Epoch: 0, Step: 100, Loss: 7.303182125091553
Epoch: 0, Step: 150, Loss: 6.423935413360596
Epoch: 0, Step: 200, Loss: 6.561067581176758
Epoch: 0, Step: 250, Loss: 6.342402935028076
Epoch: 0, Step: 300, Loss: 5.706963062286377
Epoch: 0, Step: 350, Loss: 7.268219470977783
Epoch: 0, Step: 400, Loss: 7.176136493682861
Epoch: 0, Step: 450, Loss: 6.8507561683654785
Epoch: 0, Step: 500, Loss: 8.896492004394531
Epoch: 0, Step: 550, Loss: 5.609621524810791
Epoch: 0, Step: 600, Loss: 6.651226043701172
Epoch: 0, Step: 650, Loss: 7.204199314117432
Epoch: 0, Step: 700, Loss: 6.558576583862305


Token indices sequence length is longer than the specified maximum sequence length for this model (716 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (781 > 512). Running this sequence through the model will result in indexing errors


Epoch: 0, Step: 750, Loss: 6.090606689453125
Epoch: 0, Step: 800, Loss: 7.1193623542785645
Epoch: 0, Step: 850, Loss: 5.519350528717041
Epoch: 0, Step: 900, Loss: 8.048646926879883
Epoch: 0, Step: 950, Loss: 5.735588550567627
Epoch: 0, Step: 1000, Loss: 5.848239421844482


In [25]:
idx = 0
trg = [ds.ch_tokenizer.cls_token_id] + [ds.ch_tokenizer.sep_token_id] * len(ds[idx]["zh"])
txt, l = generate(ds[idx]["en"], trg)

In [26]:
l

tensor([   0, 6821, 3221,  671,  702, 8024, 2769,  812, 4638, 4638, 4638, 8024,
        2769,  812, 4638, 4638, 4638, 4638, 8024, 2769,  812, 4638, 4638, 4638,
        4638, 8024, 2769,  812, 4638, 4638, 4638, 4638, 8024, 2769,  812, 4638,
        4638, 4638, 4638, 8024, 2769,  812, 4638, 4638, 4638,  812, 4638, 4638,
        4638,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       device='mps:0')

In [27]:
txt

'[PAD] 这 是 一 个 ， 我 们 的 的 的 ， 我 们 的 的 的 的 ， 我 们 的 的 的 的 ， 我 们 的 的 的 的 ， 我 们 的 的 的 的 ， 我 们 的 的 的 们 的 的 的 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'